In [1]:
import warnings
warnings.filterwarnings('ignore')

import gc
import re
import string
import operator
from collections import defaultdict

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import matplotlib.pyplot as plt
import seaborn as sns

#import tokenization
#from wordcloud import STOPWORDS

from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import precision_score, recall_score, f1_score

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import Dense, Input, Dropout, GlobalAveragePooling1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback


SEED = 1337

print(1)

1


In [2]:
train = pd.read_csv('./m1/train.csv', dtype={'id': np.int16, 'target': np.int8})
test = pd.read_csv('./m1/test.csv', dtype={'id': np.int16, 'target': np.int8})
# d1000 = dfull[:1000]
# d10000 = dfull[:10000]
# d100000 = dfull[:100000]


print('Training Set Shape = {}'.format(train.shape))
print('Training Set Memory Usage = {:.2f} MB'.format(train.memory_usage().sum() / 1024**2))

Training Set Shape = (27481, 4)
Training Set Memory Usage = 0.84 MB


In [3]:
train.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [4]:
train["textID"].nunique()

27481

In [5]:
print(train.shape)
print(test.shape)

(27481, 4)
(3534, 3)


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27481 entries, 0 to 27480
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         27481 non-null  object
 1   text           27480 non-null  object
 2   selected_text  27480 non-null  object
 3   sentiment      27481 non-null  object
dtypes: object(4)
memory usage: 858.9+ KB


In [7]:
train.dropna(inplace=True)

In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3534 entries, 0 to 3533
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   textID     3534 non-null   object
 1   text       3534 non-null   object
 2   sentiment  3534 non-null   object
dtypes: object(3)
memory usage: 83.0+ KB


In [9]:
train.describe()

,textID,text,selected_text,sentiment
count,27480,27480,27480,27480
unique,27480,27480,22463,3
top,a05d814df2,One of these times I`m going to have the self-...,good,neutral
freq,1,1,199,11117


In [10]:
temp = train.groupby('sentiment').count()['text'].reset_index().sort_values(by='text',ascending=False)
temp.style.background_gradient(cmap='Purples')

,sentiment,text
1,neutral,11117
2,positive,8582
0,negative,7781


In [13]:
train['Num_words_ST'] = train['selected_text'].apply(lambda x:len(str(x).split())) #Number Of words in Selected Text
train['Num_word_text'] = train['text'].apply(lambda x:len(str(x).split())) #Number Of words in main text
train['difference_in_words'] = train['Num_word_text'] - train['Num_words_ST'] #Difference in Number of words text and Selected Text



0        False
1        False
2        False
3        False
4        False
         ...  
27476    False
27477    False
27478    False
27479    False
27480    False
Name: Num_word_text, Length: 27480, dtype: bool


In [14]:
k = train[train['Num_word_text']<=2]



In [16]:
print(train[train['Num_word_text']<=2])

           textID                  text         selected_text sentiment  Num_words_ST  Num_word_text  difference_in_words
7      50e14c0bb8            Soooo high            Soooo high   neutral             2              2                    0
26     852edc3769            I`m sorry.            I`m sorry.  negative             2              2                    0
42     2e7082d1c8              MAYDAY?!              MAYDAY?!   neutral             1              1                    0
61     a4b0888da6              haha yes              haha yes   neutral             2              2                    0
68     fa2654e730              Chilliin              Chilliin  positive             1              1                    0
...           ...                   ...                   ...       ...           ...            ...                  ...
27252  a68443a78b             This year             This year   neutral             2              2                    0
27281  adb7ab12f3       